In [1]:
# !pip install --upgrade langchain
#!pip install --upgrade langchain-groq
# !pip install -U langchain-community

In [2]:
import os
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFDirectoryLoader, PyPDFLoader, WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS
from langchain.agents.agent_toolkits import ZapierToolkit

from langchain.utilities.zapier import ZapierNLAWrapper
from langchain.agents import initialize_agent , AgentType
import random
from groq import Groq
import streamlit as st
api_keys = ['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','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',
            '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','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']

client = Groq(
    
    api_key = random.choice(api_keys)
)

llm = ChatGroq(groq_api_key = client.api_key,
               model_name = "llama3-70b-8192")

In [3]:
from cryptography.fernet import Fernet
import base64
from dotenv import load_dotenv
import os
# Load environment variables from .env
load_dotenv()
with open('C:\\Users\\pc-user1\\Desktop\\key.txt', 'r') as file:
    k = file.read().strip()[2:-1]
    #k = base64.urlsafe_b64decode(k.encode())

print(k)
# Decode and decrypt the API key
encoded_encrypted_api_key = os.getenv("ENCRYPTED_API_KEY")
encrypted_api_key = base64.b64decode(encoded_encrypted_api_key)
decipher_suite = Fernet(k)


3BPcoXbY159JUXCcmaGtWxs2dEYXyzWgB0AQqAjtkCo=


In [4]:
zapier = ZapierNLAWrapper(zapier_nla_api_key=decipher_suite.decrypt(encrypted_api_key).decode())
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
agent = initialize_agent(toolkit.tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [5]:
for tool in toolkit.tools:
    print(tool.name)
    print(tool.description)
    print("\n\n")

Google Docs: Append Text to Document
A wrapper around Zapier NLA actions. The input to this tool is a natural language instruction, for example "get the latest email from my bank" or "send a slack message to the #general channel". Each tool will have params associated with it that are specified as a list. You MUST take into account the params when creating the instruction. For example, if the params are ['Message_Text', 'Channel'], your instruction should be something like 'send a slack message to the #general channel with the text hello world'. Another example: if the params are ['Calendar', 'Search_Term'], your instruction should be something like 'find the meeting in my personal calendar at 3pm'. Do not make up params, they will be explicitly specified in the tool description. If you do not have enough information to fill in the params, just say 'not enough information provided in the instruction, missing <param>'. If you get a none or null response, STOP EXECUTION, do not try to an

In [10]:
agent.invoke({"input":"""
Can you please put ALL OF THE FOLLOOWING the following into a formatted google document for a renovation quote contract? IF ONE EXISTS WITH THE SAME TITLE YOU WANT TO PUT, YOU MUST MODIFY THAT EXISTING DOC
YOU MUST FORMAT IT PROPERLY BEFORE WRITING IT INTO DOC.
          THE TEXT: 'RENOVATION QUOTE

**Project Overview:**
- Project Description: Basement renovation project in Herndon, VA 20170
- Length of Time for Project: Not specified

**Cost Breakdown:**

1. **Labor Costs:**
   - Total Labor Cost: $45,000
   
2. **Material Costs:**
   - Total Material Cost: $33,500
   - Itemized Costs:
     + Flooring: $5,000
     + Lighting: $3,000
     + Shelving: $3,500
     + Mirror and Decor: $1,000
     + Seating and Furniture: $4,000
     + Hot Tub and Glass Wall: $8,000
     + Electrical and HVAC: $6,000
     + Miscellaneous: $3,000

**Total Estimated Cost:**
- Total Cost of Labor: $45,000
- Total Cost of Materials: $33,500
- **Grand Total: $78,500**

**Payment Terms:**
- Deposit Required: 20% of the total estimated cost ($15,700.00)
- Payment Schedule: Monthly payments for 6 months, with the final payment due upon completion of the project
- Monthly Payment: $10,466
- Final Payment Due: Upon completion of the project'

"""})



> Entering new AgentExecutor chain...
Thought: I understand that I need to create a Google Document with a specific title, and if a document with the same title already exists, I need to modify that existing document. I will format the provided text properly before writing it into the document.

Action: Google Docs: Create Document from Text
Action Input: 'RENOVATION QUOTE', 'Project Overview:\n- Project Description: Basement renovation project in Herndon, VA 20170\n- Length of Time for Project: Not specified\n\n**Cost Breakdown:\n1. **Labor Costs:**\n   - Total Labor Cost: $45,000\n\n2. **Material Costs:**\n   - Total Material Cost: $33,500\n   - Itemized Costs:\n     + Flooring: $5,000\n     + Lighting: $3,000\n     + Shelving: $3,500\n     + Mirror and Decor: $1,000\n     + Seating and Furniture: $4,000\n     + Hot Tub and Glass Wall: $8,000\n     + Electrical and HVAC: $6,000\n     + Miscellaneous: $3,000\n\n**Total Estimated Cost:**\n- Total Cost of Labor: $45,000\n- Total Cost 

c:\DEV\WebdevFolder\RealEstateAI\.venv\Lib\site-packages\langchain_community\tools\zapier\tool.py:135: LangChainDeprecationWarning: This tool will be deprecated on 2023-11-17. See https://nla.zapier.com/sunset/ for details
  warn_deprecated(



Observation: {"title": "RENOVATION QUOTE", "embedLink": "https://docs.google.com/document/d/1J21s65tU4mXS7OJCbyy9mV9MaRTAcgBZMnCgboxzlYo/preview?ouid=115521521832671770773", "id": "1J21s65tU4mXS7OJCbyy9mV9MaRTAcgBZMnCgboxzlYo", "createdDate": "2024-05-19T02:28:48.961Z", "ownerNames": ["Chris John"], "mimeType": "application/vnd.google-apps.document", "editable": true, "kind": "drive#file", "modifiedDate": "2024-05-19T02:28:49.010Z", "copyRequiresWriterPermission": false, "selfLink": "https://www.googleapis.com/drive/v2/files/1J21s65tU4mXS7OJCbyy9mV9MaRTAcgBZMnCgboxzlYo", "alternateLink": "https://docs.google.com/document/d/1J21s65tU4mXS7OJCbyy9mV9MaRTAcgBZMnCgboxzlYo/edit?usp=drivesdk", "fileSize": "1", "spaces": ["drive"], "owners": [{"displayName": "Chris John", "kind": "drive#user", "isAuthenticatedUser": true, "permissionId": "11283654959933964771", "emailAddress": "thegreatchrisjohn@gmail.com", "picture": {"url": "https://lh3.googleusercontent.com/a/ACg8ocJy5oq-m0etxDBbvGdHX3Ll2g

{'input': "\nCan you please put ALL OF THE FOLLOOWING the following into a formatted google document for a renovation quote contract? IF ONE EXISTS WITH THE SAME TITLE YOU WANT TO PUT, YOU MUST MODIFY THAT EXISTING DOC\nYOU MUST FORMAT IT PROPERLY BEFORE WRITING IT INTO DOC.\n          THE TEXT: 'RENOVATION QUOTE\n\n**Project Overview:**\n- Project Description: Basement renovation project in Herndon, VA 20170\n- Length of Time for Project: Not specified\n\n**Cost Breakdown:**\n\n1. **Labor Costs:**\n   - Total Labor Cost: $45,000\n   \n2. **Material Costs:**\n   - Total Material Cost: $33,500\n   - Itemized Costs:\n     + Flooring: $5,000\n     + Lighting: $3,000\n     + Shelving: $3,500\n     + Mirror and Decor: $1,000\n     + Seating and Furniture: $4,000\n     + Hot Tub and Glass Wall: $8,000\n     + Electrical and HVAC: $6,000\n     + Miscellaneous: $3,000\n\n**Total Estimated Cost:**\n- Total Cost of Labor: $45,000\n- Total Cost of Materials: $33,500\n- **Grand Total: $78,500**\n\